# Peripheral showcase: Sensors in CHI@Edge

Welcome to this Jupyter notebook guide on using sesnors in CHI@Edge.

We currently have two types of "sense hats":
* [Raspberry Pi Sense Hat](https://www.raspberrypi.com/documentation/accessories/sense-hat.html)
* [Waveshare Sense Hat (B)](https://www.waveshare.com/wiki/Sense_HAT_(B))

These sense hats each package a 3 axis gyroscope, accelerometer, and magnetometer, as well as a variety of environmental sensors for temperature, pressure, and humidity. 

This artifact will walk you through the steps to access and utilize a sense hat for your edge computing experiments, as well as the differences between the two types.

Please check out our extensive [documentation](https://chameleoncloud.gitbook.io/chi-edge/device-enrollment/peripherals-and-device-profiles) on how to add peripherals to CHI@Edge devices.

In the following example we'll show how to use the sense hat to capture sensor data.

This notebook will demonstrate the RPi sense hat, and [waveshare_sense.ipynb](waveshare_sense.ipynb) the waveshare sense hat.

## Authenticate to CHI@Edge

As with our other artifacts, python-chi has some helper methods to connect more easily when run from inside jupyterhub.

If you're running this artifacts somewhere else, such as on your laptop, please note the commented section!


In [ ]:
import chi

chi.context.use_site("CHI@Edge")
chi.context.choose_project() # comment this line if running outside of jupyterhub

# # If running this nodebook outside of chameleon's jupyterhub, uncomment this section and fill in your_project_name
# import openstack
# chi.set("project_domain_name", "chameleon")
# chi.set("project_name", "your_project_name")
# connection = openstack.connect(cloud="edge")
# chi.set("token", connection.auth_token)

## Creating the Lease

To access the sense hat, we need to make a lease for a specific device that the camera is currently attached to.

In [ ]:
from chi import lease
import os

# machine name refers to the "type" of device
machine_name = "raspberrypi4-64"

# Reserving the specific device to which the Sense Hat is attached
# Note: Soon, we'll have peripherals listed in the hardware browser, and reservable via blazar query, so you won't need to know the right name ahead of time

# device_name = "iot-rpi4-picam2" # RPI sense hat
device_name = "iot-rpi4-picam3" # RPI sense hat
# device_name = "iot-rpi-cm4-02" # waveshare sense hat

# make a unique name for the lease
username = os.environ.get("USER")
lease_name = f"{username}-{device_name}"
container_lease = lease.Lease(name=lease_name, duration=lease.timedelta(hours=4))
container_lease.add_device_reservation(
    amount=1, machine_type=machine_name, device_name=device_name
)
container_lease.submit()
print("Done!")

## Accessing sensors with sense_hat library
Raspberry Pi Provides a well documented library to interact with the sense hat, appropriatley named "sense_hat. See https://sense-hat.readthedocs.io/en/latest/

The following will launch a container with the needed libraries built in.


In [ ]:
from chi import container

# Set a name for the container. Because CHI@Edge uses Kubernetes, ensure that underscores aren't in the name
container_name = "edge-rpi-sensehat".replace("_", "-")

environment_vars = {}

my_container = container.Container(
    name=container_name,
    image_ref="ghcr.io/chameleoncloud/edge-sensehat-image:latest",
    device_profiles=["pi_sensehat"],
    environment=environment_vars,
    reservation_id=container_lease.device_reservations[0]["id"],
    command=["-c", "sleep infinity"],
)
my_container.submit()


### The RPI sense hat provides the following sensors:

* Pressure/temperature sensor: STMicro LPS25HB
  * 260 to 1260 hPa absolute pressure range
  * 24-bit pressure data output
  * 0 to +65°C accurate temperature measurement range (±2°C)
  * 16-bit temperature data output
* Humidity temperature sensor: STMicro HTS221
  * 0 to 100% relative humidity range
  * 15 to 40°C accurate temperature measurement range (±0.5°C)
  * 16-bit data output
*  IMU: STMicro LSM9DS1 Accelerometer/gyroscope/magnetometer
  * ±16g acceleration measurement range
  * ±16 gauss magnetometer measurement range
  * ±2000dps gyroscope measurement range
  * 16-bit resolution for each measurement channel
* Color sensor: TCS3400 RGB colour and brightness sensor
* LED matrix: 8×8 RGB LED display
* Joystick: 5-button miniature joystick with up, down, left, right, and middle-click

The following example will get readings from both the pressure and humidity sensors, (and temperature from both of them).

In [ ]:
cmd_str = """
from sense_hat import SenseHat

sense = SenseHat()
sense.clear()

# Returns the pressure in Millibars
pressure = sense.get_pressure()

# Returns the temperature in Celsius from the pressure sensor
temperature_pressure = sense.get_temperature_from_pressure()

# Returns the percentage of relative humidity
humidity = sense.get_humidity()

# Returns the temperature in Celsius from the humidity sensor
temperature_humidity = sense.get_temperature_from_humidity()

print(
    "Pressure: {:.1f} mbar, Temperature (from pres): {:.1f} C,  Relative Humidity: {:.1f} %, Temperature (from hum): {:.1f} C".format(
        pressure, temperature_pressure, humidity, temperature_humidity
    )
)
"""

# if you see '0' values, try running this again. The sensors may need to be initialized.

stdout,resultcode = my_container.execute(f"python3 -c '{cmd_str}'")
print(stdout)


## Debugging, and poking around at a lower level

The sense hat exposes most sensors over i2c, but the display will appear under `/dev/fb0`, and the joystick under `/dev/input`.
For the most part, RPI's [RTIMULib](https://github.com/RPi-Distro/RTIMULib) abstracts this for you, but you're free to poke around.


### Displaying the I2C sensor map and understanding the mapping of I2C device addresses to sensors
We use the ```i2cdetect``` command from the ```i2c-tools``` utility to display the current detected i2c devices. To understand the mapping of these addresses to the various sensors onboard the sense hat, we end up needing to look up datasheets for each sensor mentioned in [Raspberry Pi's datasheet for the sense hat](https://datasheets.raspberrypi.com/sense-hat/sense-hat-product-brief.pdf) and [top level documentation](https://www.raspberrypi.com/documentation/accessories/sense-hat.html)

### Sensor I2C device addresses

- LPS22HB: 0x5C # pressure sensor

- 0x1c: LSM9DS1  -- IMU (Magnetometer)
- 0x39: TCS3400  -- Color and Brightness
- 0x46: LED2472G -- shift register for LED Matrix
- 0x5c: LPS25H   -- Pressure/Temperature sensor
- 0x5f: HTS221   -- Humidity/Temperature sensor
- 0x6a: LSM9DS1  -- IMU (Accelerometer and Gyroscope)
- 0x46: eeprom for the sense hat

In [ ]:
# example showing detected i2c addresses
result, code = my_container.execute("i2cdetect -y 1")
print(result)

### Diving deeper

We won't get into the weeds with the Pi sense hat, but check out the [waveshare sense hat](waveshare_sense.ipynb) example for a lower level guide.

## Cleaning up

In [ ]:
# delete the container so it doesn't hang around
my_container.delete()

In [ ]:
# deleting the lease will free up the device now, instead of waiting for it to expire later
container_lease.delete()